In [1]:
import torch
import numpy as np
from scipy.io import loadmat
from BCG_ichol import BCG, ichol
from scipy.sparse.linalg import inv
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import cg

## Point estimation via BCG

In [2]:
A_csc = loadmat('sparse_matrix.mat')['spmat']

In [3]:
A = torch.tensor(A_csc.toarray(), dtype=torch.double)

In [4]:
L = ichol(A)

In [5]:
print(((L.mm(L.t()) - A)**2).sum()/((A**2).sum()))

tensor(0.1398, dtype=torch.float64)


In [6]:
L = csc_matrix(L.data.numpy())

In [7]:
P = L.dot(L.T)

In [8]:
print(np.linalg.norm(P.toarray()-A_csc.toarray()))

5.27075007834948


In [16]:
torch.manual_seed(100);

In [17]:
x = torch.randn((A.shape[0],1), dtype=torch.double)
b = A.mm(x)

In [18]:
x_numpy = x.data.numpy().reshape(100,)

## Choose preconditioner

In [19]:
invP = inv(P).toarray()
invA = inv(A_csc).toarray()
eye_mat = np.eye(100)

In [20]:
bcg = BCG(A, b, torch.zeros(100, 1, dtype=torch.double), torch.tensor(eye_mat, dtype=torch.double), 
          10^(-5), 100, detailed=True, batch_directions=False)

In [21]:
result, sigma_F, nu, _, _ = bcg.bcg()

In [22]:
for i in range(100):
    print(np.linalg.norm(result[i].data.numpy().reshape(100,) - x_numpy)/np.linalg.norm(x_numpy))

0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789255407
0.4493867789

## Point Estimation via CG

In [19]:
class cg_callback(object):
    def __init__(self):
        self.x_m = []
    def __call__(self, xk=None):
        self.x_m.append(xk)

In [20]:
cg_call = cg_callback()

In [24]:
x_res = cg(A_csc, b.data.numpy(), x0=np.ones((100,)), maxiter=100, tol=10^(-10), callback=cg_call)

In [31]:
for i in range(100):
    print(np.linalg.norm(cg_call.x_m[i] - x_numpy)/np.linalg.norm(x_numpy)) 

1.128147151915888
1.0342689882530511
1.01360179897557
1.0056988471771877
1.0028423373589925
1.001980238427534
1.0013178040752302
1.000893620054404
1.0006870207233334
1.0004489221350645
1.0003117677129767
1.0002443224383226
1.0002060160711221
1.000171805432791
1.000149565684846
1.0001203356571058
1.000085185816216
1.0000624851710738
1.0000433044217383
1.000028117945125
1.0000227793368728
1.000017256638964
1.0000111756779546
1.000006727015676
1.0000029492564917
1.000000724822386
0.9999989979176019
0.9999973626509693
0.99999594667829
0.9999944466514921
0.999993383931789
0.999992944883271
0.9999926101737925
0.999992409530679
0.999992345409657
0.9999924257188204
0.9999927657939879
0.9999930479310432
0.9999932525623065
0.9999934132139983
0.9999934924468549
0.9999936045275237
0.99999377073953
0.999994039760036
0.9999943422798858
0.9999945008971439
0.9999947001849047
0.9999948809786229
0.9999950632931109
0.9999953312483993
0.9999956523004632
0.9999959828149311
0.999996330372583
0.9999966985131